In [3]:
import pandas as pd 
from datasets import load_dataset, load_from_disk

In [4]:
meta = load_dataset("parquet", 
                    data_files="/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet")["train"]

Generating train split: 184146434 examples [00:20, 8911737.66 examples/s] 


In [5]:
ds = load_dataset(
    "parquet",
    data_files="/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet",
    split="train",
    streaming=True
)


In [6]:
stream = load_dataset(
    "parquet",
    data_files="/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet",
    streaming=True
)

In [7]:
from datasets import Dataset

ds = Dataset.from_parquet("/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet")

Generating train split: 184146434 examples [00:18, 9966633.86 examples/s] 


In [8]:
from datasets import load_dataset

ds = load_dataset(
    "parquet",
    data_files="/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet",
    split="train",      # required if not streaming
    streaming=False     # or just omit this arg
)

print(ds)
print(ds.features)


Dataset({
    features: ['cluster_id', 'representative_id', 'member_count_xml_file', 'member_count', 'members'],
    num_rows: 184146434
})
{'cluster_id': Value(dtype='string', id=None), 'representative_id': Value(dtype='string', id=None), 'member_count_xml_file': Value(dtype='int32', id=None), 'member_count': Value(dtype='int32', id=None), 'members': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}


In [9]:
import pyarrow.parquet as pq
from datasets import Dataset

table = pq.read_table("/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet")
ds = Dataset(table)
print(ds)

Dataset({
    features: ['cluster_id', 'representative_id', 'member_count_xml_file', 'member_count', 'members'],
    num_rows: 184146434
})


In [10]:
import pyarrow.parquet as pq

table = pq.read_table("/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet")
table

pyarrow.Table
cluster_id: string
representative_id: string
member_count_xml_file: int32
member_count: int32
members: list<element: string>
  child 0, element: string
----
cluster_id: [["UniRef90_UPI002E2621C6","UniRef90_UPI00358F51CD","UniRef90_UPI00398E31D8","UniRef90_A0A5A9P0L4","UniRef90_A0ABD1JBH0",...,"UniRef90_UPI00225B580A","UniRef90_UPI00254460F6","UniRef90_UPI002796F535","UniRef90_UPI002896634E","UniRef90_UPI003D7BF59D"],["UniRef90_UPI00222E980F","UniRef90_UPI0024785306","UniRef90_UPI0035A280ED","UniRef90_UPI001B351495","UniRef90_UPI00222EC2CE",...,"UniRef90_UPI00248289A7","UniRef90_UPI0012FEA33D","UniRef90_A0A8C5NYK4","UniRef90_UPI0025ADAEBD","UniRef90_UPI003BF9D085"],...,["UniRef90_A0A3M7PZG0","UniRef90_Q9BM67","UniRef90_Q9BM66","UniRef90_E9BBM2","UniRef90_Q9BLZ4",...,"UniRef90_A0A0T5Z569","UniRef90_D4J9Q7","UniRef90_A0A6A7YBE0","UniRef90_A0A0T5Z260","UniRef90_A0A5C6IVY6"],["UniRef90_Q725V0","UniRef90_A0A4R4SIP9","UniRef90_A0ABW1B1Z7","UniRef90_A0A943LZD8","UniRef90_A0ABV6HH

In [11]:
pq.read_metadata("/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet").num_rows

184146434

In [12]:
table.schema

cluster_id: string
representative_id: string
member_count_xml_file: int32
member_count: int32
members: list<element: string>
  child 0, element: string

In [13]:
import pandas as pd
df = pd.read_csv("/gpfs/data/brandeslab/Data/uniref/uniref90_cluster_members.tsv", sep="\t", nrows=100)
df

,ClusterID,RepresentativeID,member_count_xml_file,member_count,MemberIDs
0,UniRef90_UPI002E2621C6,UPI002E2621C6,1,1,NaN
1,UniRef90_UPI00358F51CD,UPI00358F51CD,1,1,NaN
2,UniRef90_UPI00398E31D8,UPI00398E31D8,1,1,NaN
3,UniRef90_A0A5A9P0L4,A0A5A9P0L4_9TELE,1,1,NaN
4,UniRef90_A0ABD1JBH0,A0ABD1JBH0_9TELE,1,1,NaN
...,...,...,...,...,...
95,UniRef90_A0A6P8RG45,A0A6P8RG45_GEOSA,1,1,NaN
96,UniRef90_UPI003D69693D,UPI003D69693D,1,1,NaN
97,UniRef90_UPI003EBDD6D2,UPI003EBDD6D2,3,3,"UPI003EBC14C6,UPI003EB9A168"
98,UniRef90_A0A1V4K6M4,A0A1V4K6M4_PATFA,562,562,"A0A7K4RU14_COLPI,A0A7L4G2H3_9COLU,A0A094KAD8_A..."


In [14]:
df.iloc[97]

ClusterID                     UniRef90_UPI003EBDD6D2
RepresentativeID                       UPI003EBDD6D2
member_count_xml_file                              3
member_count                                       3
MemberIDs                UPI003EBC14C6,UPI003EB9A168
Name: 97, dtype: object

In [15]:
!pip install parasail

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
import parasail

In [17]:
import sys
!{sys.executable} -m pip install parasail

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Split
from transformers import PreTrainedTokenizerFast

# Define vocabulary: ONLY standard 20 amino acids
amino_acids = list("ACDEFGHIKLMNPQRSTVWY")  # Standard 20 only
special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "-"]

# Build vocabulary
vocab = {token: i for i, token in enumerate(special_tokens + amino_acids)}

print(f"Vocab: {vocab}")
print(f"'-' ID: {vocab['-']}")

# Create tokenizer
tokenizer = Tokenizer(WordLevel(vocab=vocab, unk_token="[UNK]"))
tokenizer.pre_tokenizer = Split(pattern="", behavior="isolated")

# Wrap it
hf_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]"
)

# Test
print("\nTests:")
print("'-' converts to:", hf_tokenizer.convert_tokens_to_ids("-"))
print("Tokenization of 'ACD-GH':", hf_tokenizer("ACD-GH"))
print("Decoding:", hf_tokenizer.decode([6, 7, 8, 5, 11, 12]))
print("Vocab size:", len(hf_tokenizer))

# hf_tokenizer.save_pretrained("phylo_char_tokenizer")

Vocab: {'[PAD]': 0, '[UNK]': 1, '[CLS]': 2, '[SEP]': 3, '[MASK]': 4, '-': 5, 'A': 6, 'C': 7, 'D': 8, 'E': 9, 'F': 10, 'G': 11, 'H': 12, 'I': 13, 'K': 14, 'L': 15, 'M': 16, 'N': 17, 'P': 18, 'Q': 19, 'R': 20, 'S': 21, 'T': 22, 'V': 23, 'W': 24, 'Y': 25}
'-' ID: 5

Tests:
'-' converts to: 5
Tokenization of 'ACD-GH': {'input_ids': [6, 7, 8, 5, 11, 12], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}
Decoding: A C D - G H
Vocab size: 26


In [19]:
import parasail 

In [20]:
import sys
print(sys.executable)

/gpfs/home/as12267/.conda/envs/huggingface_bert/bin/python


In [21]:
import sys
[p for p in sys.path if "conda" in p]

['/gpfs/home/as12267/.conda/envs/huggingface_bert/lib/python310.zip',
 '/gpfs/home/as12267/.conda/envs/huggingface_bert/lib/python3.10',
 '/gpfs/home/as12267/.conda/envs/huggingface_bert/lib/python3.10/lib-dynload',
 '/gpfs/home/as12267/.conda/envs/huggingface_bert/lib/python3.10/site-packages']

In [22]:
import parasail
result = parasail.nw_trace_scan_16("AAAA", "AAAA", 5, 1, parasail.blosum62)

print(dir(result))

['__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_as_parameter_', '_cigar', '_traceback', '_traceback_args', 'cigar', 'end_query', 'end_ref', 'get_cigar', 'get_traceback', 'len_query', 'len_ref', 'length', 'length_col', 'length_row', 'length_table', 'matches', 'matches_col', 'matches_row', 'matches_table', 'matrix', 'pointer', 'query', 'ref', 'saturated', 'score', 'score_col', 'score_row', 'score_table', 'similar', 'similar_col', 'similar_row', 'similar_table', 'traceback']


In [23]:
import parasail

matrix = parasail.blosum62

def align_pair(seq1, seq2):
    # global Needleman–Wunsch alignment with traceback
    result = parasail.nw_trace_scan_16(seq1, seq2, 10, 1, matrix)

    # Extract the traceback object
    tb = result.traceback

    # Get aligned sequences
    # a1 = tb.query.replace("-", "<GAP>")
    # a2 = tb.ref.replace("-", "<GAP>")
    a1 = tb.query
    a2 = tb.ref

    return a1, a2

# Test sequences
seq1 = "MKTAYIAKQRQISFVKSHFSRQLEERLGLIE"
seq2 = "MKTVYIAKRRQISFLKSHFSRQLDDRLGLIE"

a1, a2 = align_pair(seq1, seq2)

print("Seq1 Aligned:", a1)
print("Seq2 Aligned:", a2)


Seq1 Aligned: MKTAYIAKQRQISFVKSHFSRQLEERLGLIE
Seq2 Aligned: MKTVYIAKRRQISFLKSHFSRQLDDRLGLIE


In [24]:
seq1 = "MKTAYIAKQRQISFVKSHFS"
seq2 = "MKTA---QRQISFVKSHFSSS"

a1, a2 = align_pair(seq1, seq2)

print("Seq1 Aligned:", a1)
print("Seq2 Aligned:", a2)

Seq1 Aligned: MKTAYIAKQRQISFVKSHF--S
Seq2 Aligned: MKTA----QRQISFVKSHFSSS


In [25]:
from transformers import (
    PreTrainedTokenizerFast,
)

class PhyloTokenizerLoader:
    def __init__(self, tokenizer_path):
        self.tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_path)
    
    def encode(self, aligned_seq1, aligned_seq2):
        inputs = self.tokenizer(
            aligned_seq2,
            add_special_tokens=False
        )
        labels = self.tokenizer(
            aligned_seq1,
            add_special_tokens=False
        )
        return{
            "input_ids": inputs["input_ids"],
            "labels": labels["input_ids"], 
            "attention_mask": inputs["attention_mask"]
        }
    
    def __call__(self, *args, **kwargs):
        return self.tokenizer(*args, **kwargs)

# Set breakpoint inside encode() method
# Then run this:
phylo_tokenizer = PhyloTokenizerLoader("phylo_char_tokenizer")

# When you call encode, debugger will pause at your breakpoint
result = phylo_tokenizer.encode("ATCG", "ATCG")

In [26]:
result

{'input_ids': [6, 22, 7, 11],
 'labels': [6, 22, 7, 11],
 'attention_mask': [1, 1, 1, 1]}

In [27]:
from gLM.dataset import UniRefClusterIterableDataset
from gLM.tokenizers import PhyloTokenizerLoader


In [28]:
tokenizer = PhyloTokenizerLoader("./phylo_char_tokenizer").load()


In [29]:
ds = UniRefClusterIterableDataset(
    parquet_path="/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet",
    fasta_path="/gpfs/data/brandeslab/Data/uniref/uniref100.fasta",
    tokenizer=tokenizer,
    max_seq_len=8192,
)



TypeError: UniRefClusterIterableDataset.__init__() missing 1 required positional argument: 'index_db_path'

In [ ]:
for i, x in enumerate(ds):
    print("Got item", i)
    if i == 2:
        break


In [ ]:

import time
from gLM.dataset import UniRefClusterIterableDataset
from gLM.tokenizers import PhyloTokenizerLoader

parquet = "/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet"
fasta   = "/gpfs/data/brandeslab/Data/uniref/uniref100.fasta"

print("Loading tokenizer...")
tok = PhyloTokenizerLoader("./phylo_char_tokenizer").load()

print("Instantiating dataset...")
start = time.time()
ds = UniRefClusterIterableDataset(
    parquet_path=parquet,
    fasta_path=fasta,
    tokenizer=tok,
    max_seq_len=8192,
)
print("Dataset constructed in", time.time() - start, "sec")

print("Fetching first item...")
start = time.time()
for i, item in enumerate(ds):
    print("Got first item!")
    break
print("First item fetched in", time.time() - start, "sec")



In [ ]:
from gLM.dataset import UniRefClusterIterableDataset
from gLM.tokenizers import PhyloTokenizerLoader
import time

parquet = "/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet"
fasta   = "/gpfs/data/brandeslab/Data/uniref/uniref100.fasta"
index_db = "/gpfs/data/brandeslab/User/as12267/uniref100.idx"

print("Loading tokenizer...")
tok = PhyloTokenizerLoader("./phylo_char_tokenizer")

ds = UniRefClusterIterableDataset(
    parquet_path=parquet,
    index_db_path=index_db,
    fasta_path=fasta,
    tokenizer=tok,
    max_seq_len=8192,
)

print("Fetching first item...")
for i, x in enumerate(ds):
    print("Success!")
    print(x.keys())
    break



/gpfs/home/as12267/.conda/envs/huggingface_bert/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading tokenizer...
Fetching first item...
REP: A0ABS9P3Y9_9GAMM MEMBERS: [] <class 'list'>
REP: A0A835XQ69_9CHLO MEMBERS: ['A0A835XMW2_9CHLO'] <class 'list'>
PAIR: A0A835XQ69_9CHLO A0A835XMW2_9CHLO
REP: UPI000C86E4D3 MEMBERS: [] <class 'list'>
REP: UPI001260CD7F MEMBERS: [] <class 'list'>
REP: A0A2I1HB31_9GLOM MEMBERS: [] <class 'list'>
REP: A0AAF0EAU2_9BASI MEMBERS: [] <class 'list'>
REP: UPI002ECFBE8A MEMBERS: [] <class 'list'>
REP: UPI0002491D06 MEMBERS: [] <class 'list'>
REP: UPI002625D505 MEMBERS: [] <class 'list'>
REP: A0A917ZIC9_9GAMM MEMBERS: [] <class 'list'>
REP: UPI00188B8639 MEMBERS: [] <class 'list'>
REP: UPI003216E2CD MEMBERS: [] <class 'list'>
REP: A0A1R1K1Z6_ALCXX MEMBERS: ['UPI0029A3963B', 'UPI0005F94960'] <class 'list'>
PAIR: UPI0005F94960 A0A1R1K1Z6_ALCXX
REP: A0ABU7LKA1_9NOCA MEMBERS: ['UPI000B257CD9', 'UPI00364A542B'] <class 'list'>
PAIR: UPI000B257CD9 A0ABU7LKA1_9NOCA
REP: UPI00112C5A48 MEMBERS: [] <class 'list'>
REP: A0A1I2CAG8_9RHOB MEMBERS: [] <class 'list'>


In [2]:
from gLM.dataset import UniRefClusterIterableDataset
from gLM.tokenizers import PhyloTokenizerLoader
from gLM.collator import SequencePairCollator
import time
from torch.utils.data import DataLoader

parquet = "/gpfs/data/brandeslab/Data/uniref/uniref90_clusters.parquet"
fasta   = "/gpfs/data/brandeslab/Data/uniref/uniref100.fasta"
index_db = "/gpfs/data/brandeslab/User/as12267/uniref100.idx"

print("Loading tokenizer...")
tok = PhyloTokenizerLoader("./phylo_char_tokenizer")

ds = UniRefClusterIterableDataset(
    parquet_path=parquet,
    index_db_path=index_db,
    fasta_path=fasta,
    tokenizer=tok,
    max_seq_len=8192,
)


collator = SequencePairCollator(pad_id=tok.tokenizer.pad_token_id)

loader = DataLoader(
    ds, 
    batch_size=4, 
    collate_fn=collator,
    num_workers=0

)
batch = next(iter(loader))

print("input_ids shape:", batch["input_ids"].shape)
print("labels shape:", batch["labels"].shape)
print("attention_mask shape:", batch["attention_mask"].shape)
print("percent_identity:", batch["percent_identity"])

Loading tokenizer...
------BATCH START------
SEQ LEN: 236
SEQ LEN: 387
SEQ LEN: 344
SEQ LEN: 121
-------BATCH END-------
input_ids shape: torch.Size([4, 387])
labels shape: torch.Size([4, 387])
attention_mask shape: torch.Size([4, 387])
percent_identity: tensor([91.9492, 91.2145, 98.2558, 98.3471])


In [7]:
tokenizer = PhyloTokenizerLoader("./phylo_char_tokenizer").load()
pad_id = tokenizer.pad_token_id
gap_id = tokenizer.convert_tokens_to_ids("-")
print("Phylo Tokenizer loaded. GAP ID:", gap_id)
print(f"vocab size: {tokenizer.vocab_size}")

Phylo Tokenizer loaded. GAP ID: 5
vocab size: 26


In [ ]:
from gLM.data_utils.uniref_cluster_sampler import RandomClusterSampler

sampler = RandomClusterSampler(parquet)
for _ in range(5):
    cluster = sampler.sample_clusters()
    print(cluster)


{'cluster_id': 'UniRef90_A0AAD2JM23', 'representative_id': 'A0AAD2JM23_9STRA', 'members': ['A0A9K3LU73_9STRA']}
{'cluster_id': 'UniRef90_A0A2T5GHF6', 'representative_id': 'A0A2T5GHF6_9SPHN', 'members': ['UPI0011B1CF18', 'UPI00177CA10F', 'UPI00335862B2', 'UPI001783B8B9', 'UPI00178503EB', 'UPI002A6B860E']}
{'cluster_id': 'UniRef90_A0ABD3RNB6', 'representative_id': 'A0ABD3RNB6_9LAMI', 'members': []}
{'cluster_id': 'UniRef90_A0ABN7WPN7', 'representative_id': 'A0ABN7WPN7_GIGMA', 'members': ['A0ABN7X348_GIGMA']}
{'cluster_id': 'UniRef90_UPI0031F96DD0', 'representative_id': 'UPI0031F96DD0', 'members': []}


In [ ]:
import parasail
print(parasail.__version__)


1.3.4


In [1]:
import pandas as pd
import numpy as np
from gLM.models.protein_bert import ProteinBertModel
from gLM.tokenizers import TokenizerLoader
from transformers import Trainer, TrainingArguments
from gLM.callbacks import ZeroShotVEPEvaluationCallback  

# 1. Create a minimal test CSV with VEP data
test_data = {
    'sequence': [
        'MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLSGAEKAVQVKVKALPDAQFEVVHSLAKWKRQTLGQHDFSAGEGLYTHMKALRPDEDRLSLEVGQSGNDIAAYRLTDMPPTMAQQVKQPLQERAQAWADTGAATRAVLPRLCGVMNHRFQTHAAVDALRLSGAVPRTLVALLASPE',
        'MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLSGAEKAVQVKVKALPDAQFEVVHSLAKWKRQTLGQHDFSAGEGLYTHMKALRPDEDRLSLEVGQSGNDIAAYRLTDMPPTMAQQVKQPLQERAQAWADTGAATRAVLPRLCGVMNHRFQTHAAVDALRLSGAVPRTLVALLASPE',
        'MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPKVKAHGKKVLGAFSDGLAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHHFGKEFTPPVQAAYQKVVAGVANALAHKYH',
        'MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPKVKAHGKKVLGAFSDGLAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHHFGKEFTPPVQAAYQKVVAGVANALAHKYH',
    ],
    'pos': [10, 20, 15, 25],
    'ref': ['Q', 'V', 'K', 'K'],
    'alt': ['E', 'A', 'E', 'R'],
    'label': [1, 0, 1, 0]  # 1 = pathogenic, 0 = benign
}

test_csv_path = './test_vep_data.csv'
pd.DataFrame(test_data).to_csv(test_csv_path, index=False)
print(f"Created test CSV at {test_csv_path}")

# 2. Initialize tokenizer and model
tokenizer = TokenizerLoader("./char_tokenizer").load()
model = ProteinBertModel(
    vocab_size=tokenizer.vocab_size, 
    tokenizer=tokenizer, 
    attn_implementation="flash_attention_2"
).build()

# 3. Create minimal training arguments (for testing, just 1 step)
training_args = TrainingArguments(
    output_dir="./test_vep_callback",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    max_steps=1,  # Just 1 step for testing
    logging_steps=1,
    save_steps=1000,  # Don't save during test
    fp16=False,  # Disable for easier debugging
    report_to="none",  # Disable wandb for testing
)

# 4. Create dummy dataset (minimal)
class DummyDataset:
    def __len__(self):
        return 4
    
    def __getitem__(self, idx):
        # Return tokenized dummy sequences
        seq = "MKTAYIAKQRQISFVK"
        tokens = tokenizer(seq, return_tensors="pt", truncation=True, max_length=512)
        return {k: v.squeeze(0) for k, v in tokens.items()}

train_ds = DummyDataset()

# 5. Initialize the VEP callback
vep_callback = ZeroShotVEPEvaluationCallback(
    tokenizer=tokenizer,
    input_csv=test_csv_path,
    trainer=None,  # Will be set by Trainer
    max_len=512,  # Smaller for testing
    eval_every_n_steps=1,  # Evaluate after every step for testing
    batch_size=2,
    training_type="phylo",  # or "MLM" depending on your model
)

# 6. Create trainer with callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    tokenizer=tokenizer,
    callbacks=[vep_callback],
)

# 7. Test the callback directly (without training)
print("\n=== Testing VEP callback directly ===")
vep_callback.run_vep_eval(model, step_id=0)

# 8. Optional: Run one training step to verify callback integration
# print("\n=== Testing callback during training ===")
# trainer.train()

print("\n=== Test complete ===")

You are attempting to use Flash Attention 2 without specifying a torch dtype. This might lead to unexpected behaviour


Created test CSV at ./test_vep_data.csv
Using flash_attention_2 attention


/tmp/ipykernel_3720264/3008571830.py:71: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.



=== Testing VEP callback directly ===
[Rank 0] Starting zero-shot VEP eval @ step 0


/gpfs/home/as12267/.conda/envs/huggingface_bert/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:236: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


[Rank 0] Progress: 0/4


ValueError: Default process group has not been initialized, please make sure to call init_process_group.